In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import os
import math
from scipy.stats import norm, binom
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

import yfinance as yf

from pypfopt import EfficientFrontier, risk_models, expected_returns, objective_functions, base_optimizer, black_litterman, discrete_allocation

In [2]:
#Tickers and weights taken from mid term report 

tickers =['ADI','AEIS' ,'AMAT' ,'AMD' ,'AVGO' ,'CRUS' ,'DIOD','ENPH','FLEX','IIVI','INTC','JBL','LRCX','MCHP','MRVL','MTSI','MU','MXL','NVDA','NXPI','OLED','ON','QCOM','SGH','SMTC','SYNA','TSM','TTMI','TXN','XLNX','RF']
original_best_fund = pd.read_csv("best_fund_original.csv")
original_worst_fund = pd.read_csv("worst_fund_original.csv") 
original_worst_fund.drop(original_worst_fund.tail(1).index,inplace=True) #drop cash 
weights_optimised = [0.026704,0.059727,0.009675,0.154568,0.009476,0.000071,0.029709,0.031739,0.083856,0.024400,0.105450,0.024077,0.010791,0.021304,0.021667,0.010913,0.023116,0.067754,0.025232,0.017685,0.009414,0.023142,0.036448,0.039288,0.019285,0.013199,0.013870,0.053220,0.017962,0.000000,0.016258]
weights_adjusted =  [0.0117,0.0521,0.1054,0.0000,0.0346,0.0000,0.0183,0.1354,0.0365,0.0235,0.0000,0.0510,0.0602,0.0363,0.0125,0.0522,0.0346,0.0310,0.0012,0.0463,0.0000,0.0896,0.0000,0.0244,0.0061,0.0468,0.0528,0.0015,0.0359,0.0000,0.0000]
weights_worst = list(original_worst_fund['Security\'s Percentage of the Total Net Assets']/100)
weights_best = list(original_best_fund['Security\'s Percentage of the Total Net Assets']/100)

## Question 1a part 1

In [3]:
def cal_ytd_returns(tickers, data):
    ytd_returns=[]
    for ticker in tickers:
        ytd_return = (data[ticker].iloc[-1] - data[ticker].iloc[0])/data[ticker].iloc[-1]
        ytd_returns.append(ytd_return)
                      
    return ytd_returns

### i. Best Mutual Fund

In [4]:
original_best_fund.sort_values(by='Ticker Symbol Given by the Exchange', inplace=True)
original_best_fund.reset_index(drop=True, inplace=True)
original_best_fund

,Ticker Symbol Given by the Exchange,Security's Percentage of the Total Net Assets,Number of the Security's Shares,Market Value of the Security as of eff_dt,Name of the Security Held
0,ADI,1.299999,455296,6.726088e+07,ANALOG DEVICES INC
1,AEIS,0.730000,388464,3.766935e+07,ADVANCED ENERGY INDUSTRIES INC
2,AMAT,2.129999,1282500,1.106798e+08,APPLIED MATERIALS INC
3,AMBA,0.180000,101700,9.338094e+06,AMBARELLA INC
4,AMD,5.289997,2994500,2.746256e+08,ADVANCED MICRO DEVICES INC
5,AOSL,0.340000,742296,1.754788e+07,ALPHA & OMEGA SEMICONDUCTOR LTD
6,ASX,0.160000,1396600,8.156144e+06,A S E TECHNOLOGY HLDG CO LTD
7,AVGO,4.689999,555550,2.432476e+08,BROADCOM INC
8,CRUS,2.080000,1311560,1.078102e+08,CIRRUS LOGIC INC
9,DIOD,0.910000,667300,4.704465e+07,DIODES INC


In [5]:
#finding ytd returns for original portfolio
tickers_best_ori = original_best_fund['Ticker Symbol Given by the Exchange']
price_data_best = yf.download(list(tickers_best_ori),start = '2021-01-01',end = '2021-10-31')
price_data_best_ori = price_data_best['Adj Close']   

[*********************100%***********************]  34 of 34 completed


In [6]:
price_data_best_ori

,ADI,AEIS,AMAT,AMBA,AMD,AOSL,ASX,AVGO,CRUS,DIOD,...,ON,QCOM,SEDG,SGH,SMTC,SYNA,TSM,TTMI,TXN,XLNX
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-31,145.864197,96.663727,85.829178,91.820000,91.709999,23.639999,5.649126,427.943329,82.199997,70.500000,...,32.730000,150.148392,319.119995,37.630001,72.089996,96.400002,107.772926,13.80,160.357758,141.505264
2021-01-04,144.175797,97.680496,86.396065,91.339996,92.300003,24.490000,5.774877,415.599121,84.129997,71.239998,...,32.669998,146.363663,311.350006,38.639999,71.180000,97.400002,110.402008,13.75,158.491653,142.164032
2021-01-05,146.752838,101.538284,89.111160,95.250000,92.769997,26.340000,6.113438,418.413940,86.360001,74.019997,...,33.220001,150.237091,316.809998,39.560001,72.290001,98.500000,111.459572,14.02,159.654297,143.960663
2021-01-06,147.414383,105.924385,90.334457,102.650002,90.330002,28.000000,6.219842,415.833649,90.379997,77.360001,...,34.060001,149.014954,348.130005,39.240002,73.519997,101.269997,114.266579,14.34,160.445679,140.956299
2021-01-07,153.644669,111.177742,94.044106,106.410004,95.160004,28.490000,6.355267,433.358002,91.360001,79.089996,...,35.360001,153.460068,365.970001,41.384998,74.650002,102.639999,120.018951,14.28,164.080200,149.430450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,180.520004,85.820000,135.759995,179.850006,122.360001,33.240002,6.940000,522.070007,79.370003,94.089996,...,46.220001,131.929993,311.920013,51.480000,80.180000,179.399994,113.639999,13.56,198.974518,174.982635
2021-10-26,179.880005,85.279999,132.000000,181.000000,122.930000,32.830002,7.000000,533.869995,79.760002,93.889999,...,45.139999,131.940002,313.880005,50.990002,80.540001,178.759995,114.180000,13.69,195.774231,171.559052
2021-10-27,170.460007,84.760002,132.160004,181.690002,122.279999,32.540001,6.930000,523.109985,80.320000,93.540001,...,44.549999,131.199997,342.540009,50.680000,81.370003,181.429993,113.750000,13.60,185.954712,180.512299


In [145]:
ytd_best = pd.DataFrame(tickers_best_ori)
ytd_best['ytd_returns']=cal_ytd_returns(tickers_best_ori, price_data_best_ori)
ytd_best['weights'] = original_best_fund['Security\'s Percentage of the Total Net Assets']/100 
ytd_best['weighted_ytd_returns'] = ytd_best['ytd_returns']*ytd_best['weights']

In [146]:
q1a_results = pd.DataFrame(columns=['portfolio', 'ytd returns', 'annualised volatility'])
q1a_results.loc[0] = (['best mutual fund',ytd_best['weighted_ytd_returns'].sum()*100, 0])
q1a_results

,portfolio,ytd returns,annualised volatility
0,best mutual fund,19.803784,0


### ii. Worst Mutual Fund

In [147]:
original_worst_fund.sort_values(by='Ticker Symbol Given by the Exchange', inplace=True)
original_worst_fund.reset_index(drop=True, inplace=True)
original_worst_fund

,Ticker Symbol Given by the Exchange,Security's Percentage of the Total Net Assets,Number of the Security's Shares,Market Value of the Security as of eff_dt,Name of the Security Held
0,AAPL,6.669998,64500.0,8558505.00,APPLE INC
1,ABBV,3.000000,35965.0,3853649.75,ABBVIE INC
2,ADBE,3.859999,9900.0,4951188.00,ADOBE INC
3,ADP,1.740000,12700.0,2237740.00,AUTOMATIC DATA PROCESSING INC
4,ADSK,4.160000,17500.0,5343450.00,AUTODESK INC
5,AMZN,2.539999,1000.0,3256930.00,AMAZON COM INC
6,ANTM,2.279999,9100.0,2921919.00,ANTHEM INC
7,BAH,2.109999,31000.0,2702580.00,BOOZ ALLEN HAMILTON HOLDING CORP
8,EBIX,4.219997,142500.0,5410725.00,EBIX INC
9,EEFT,4.269997,37800.0,5477976.00,EURONET WORLDWIDE INC


In [148]:
#finding ytd returns for original portfolio
tickers_worst_ori = original_worst_fund['Ticker Symbol Given by the Exchange']
price_data_worst_ori = yf.download(list(tickers_worst_ori),start = '2021-01-01',end = '2021-10-31')
price_data_worst_ori = price_data_worst_ori['Adj Close']   

[*********************100%***********************]  31 of 31 completed


In [149]:
price_data_worst_ori

,AAPL,ABBV,ADBE,ADP,ADSK,AMZN,ANTM,BAH,EBIX,EEFT,...,NXPI,PRFT,QRVO,ROG,ST,SWKS,TEL,THR,UNH,V
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-31,131.876999,102.245758,500.119995,173.686356,305.339996,3256.929932,318.126770,86.037292,37.651852,144.919998,...,157.676880,47.650002,166.270004,155.289993,52.740002,151.550049,119.765152,15.630000,346.945587,217.777298
2021-01-04,128.617111,100.585396,485.339996,166.549637,296.839996,3186.629883,315.639923,84.853027,37.265121,140.649994,...,160.681473,46.450001,164.589996,152.089996,51.970001,149.408844,119.003464,15.540000,345.778168,216.811539
2021-01-05,130.207291,101.625504,485.690002,166.086349,301.700012,3218.510010,312.192047,85.524117,37.879925,144.850006,...,164.281052,46.910000,169.250000,156.350006,53.419998,154.940308,121.288551,16.030001,341.128235,213.575684
2021-01-06,125.824326,100.747612,466.309998,165.504761,302.869995,3138.379883,325.290070,87.527512,39.972244,146.369995,...,167.305481,48.450001,172.000000,169.699997,55.470001,157.111252,124.770622,16.980000,355.444122,211.693909
2021-01-07,130.117859,101.825882,477.739990,166.441223,315.200012,3162.159912,334.266449,89.659187,39.992081,147.580002,...,173.522903,48.980000,178.570007,176.589996,56.119999,161.264786,127.273361,17.809999,361.142792,212.878723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,148.423386,108.500000,646.969971,218.070007,311.790009,3320.370117,434.559998,83.660004,31.790001,122.529999,...,200.289993,124.709999,167.320007,194.979996,56.700001,165.179993,147.690002,17.030001,449.489990,233.880005
2021-10-26,149.102402,109.489998,642.500000,218.800003,312.799988,3376.070068,436.239990,82.639999,31.760000,117.900002,...,196.149994,123.190002,165.279999,194.320007,56.119999,164.330002,146.910004,16.980000,454.640015,231.820007
2021-10-27,148.633087,108.400002,640.080017,221.949997,308.510010,3392.489990,433.679993,79.690002,30.809999,114.059998,...,192.410004,121.900002,165.270004,192.199997,54.770000,165.899994,142.839996,16.610001,453.480011,215.779999


In [150]:
ytd_worst = pd.DataFrame(tickers_worst_ori)
ytd_worst['ytd_returns']=cal_ytd_returns(tickers_worst_ori, price_data_worst_ori)
ytd_worst['weights'] = original_worst_fund['Security\'s Percentage of the Total Net Assets']/100 
ytd_worst['weighted_ytd_returns'] = ytd_worst['ytd_returns']*ytd_worst['weights']

In [151]:
ytd_worst['weighted_ytd_returns'].sum()

0.09696552795857079

In [152]:
q1a_results.loc[1] = (['worst mutual fund',ytd_worst['weighted_ytd_returns'].sum()*100, 0])
q1a_results

,portfolio,ytd returns,annualised volatility
0,best mutual fund,19.803784,0
1,worst mutual fund,9.696553,0


### iii. portfolio before subjective adjustment

In [153]:
price_data = yf.download(tickers,start = '2021-01-01',end = '2021-10-31')


[*********************100%***********************]  31 of 31 completed


In [154]:
price_data = price_data['Adj Close']
print(price_data.head(5))

                   ADI        AEIS       AMAT        AMD        AVGO  \
Date                                                                   
2020-12-31  145.864197   96.663727  85.829170  91.709999  427.943359   
2021-01-04  144.175812   97.680496  86.396065  92.300003  415.599121   
2021-01-05  146.752838  101.538277  89.111160  92.769997  418.413940   
2021-01-06  147.414368  105.924377  90.334457  90.330002  415.833679   
2021-01-07  153.644684  111.177742  94.044098  95.160004  433.358032   

                 CRUS       DIOD        ENPH       FLEX       IIVI  ...  \
Date                                                                ...   
2020-12-31  82.199997  70.500000  175.470001  17.980000  75.959999  ...   
2021-01-04  84.129997  71.239998  172.240005  17.879999  76.010002  ...   
2021-01-05  86.360001  74.019997  181.479996  17.820000  79.089996  ...   
2021-01-06  90.379997  77.360001  203.380005  18.670000  79.389999  ...   
2021-01-07  91.360001  79.089996  213.759995 

In [155]:
#to get ytd returns for optimised portfolio 
ytd_optimised = pd.DataFrame(tickers)
ytd_optimised['ytd_returns']=cal_ytd_returns(tickers, price_data)
ytd_optimised['weights'] = weights_optimised
ytd_optimised['weighted_ytd_returns'] = ytd_optimised['ytd_returns']*ytd_optimised['weights']

In [156]:
q1a_results.loc[2] = (['portfolio before subjective adjustment',ytd_optimised['weighted_ytd_returns'].sum()*100, 0])
q1a_results

,portfolio,ytd returns,annualised volatility
0,best mutual fund,19.803784,0
1,worst mutual fund,9.696553,0
2,portfolio before subjective adjustment,13.585535,0


## Question 1a part 2

In [133]:
def cal_annualised_sd(df, weights):
    return_stocks = df.pct_change()
    daily_returns_portfolio_mean = return_stocks.mean()
    
    allocated_daily_returns = (weights * daily_returns_portfolio_mean)
    portfolio_return = np.sum(allocated_daily_returns)
    
    return_stocks['portfolio_daily_returns'] = return_stocks.dot(weights)
                                                                
    matrix_covariance_portfolio = return_stocks.iloc[:,:-1]
    matrix_covariance_portfolio = (matrix_covariance_portfolio.cov())*252
    
    portfolio_variance = np.dot(np.array(weights).T,np.dot(matrix_covariance_portfolio, weights))

    portfolio_risk = np.sqrt(portfolio_variance)
    return portfolio_risk*100

### i. Best Mutual Fund

In [158]:
portfolio_volatility_i = cal_annualised_sd(price_data_best_ori, weights_best)

### ii. Worst Mutual Fund

In [136]:
portfolio_volatility_ii = cal_annualised_sd(price_data_worst_ori, weights_worst)

### iii. portfolio before subjective adjustment

In [138]:
portfolio_volatility_iii = cal_annualised_sd(price_data, weights_optimised)

In [157]:
vol = [portfolio_volatility_i, portfolio_volatility_ii, portfolio_volatility_iii]

q1a_results['annualised volatility'] = vol
q1a_results

,portfolio,ytd returns,annualised volatility
0,best mutual fund,19.803784,12.101760
1,worst mutual fund,9.696553,17.141649
2,portfolio before subjective adjustment,13.585535,20.580419


## Question 1b 

### (i) best mutual fund

In [102]:
#finding ytd returns for original portfolio
tickers_best_ori = original_best_fund['Ticker Symbol Given by the Exchange']
price_data_best_1b = yf.download(list(tickers_best_ori),start = '2021-10-01',end = '2021-10-31')
price_data_best_ori_1b = price_data_best_1b['Adj Close']   

[*********************100%***********************]  34 of 34 completed


In [107]:
monthly_best = pd.DataFrame(tickers_best_ori)
monthly_best['monthly_returns']=cal_ytd_returns(tickers_best_ori, price_data_best_ori_1b)
monthly_best['weights'] = weights_best
monthly_best['weighted_monthly_returns'] = monthly_best['monthly_returns']*monthly_best['weights']

q1b_results = pd.DataFrame(columns=['portfolio', 'total monthly returns'])
q1b_results.loc[0] = (['best mutual fund',monthly_best['weighted_monthly_returns'].sum()*100])
q1b_results

,portfolio,total monthly returns
0,best mutual fund,5.815515


### (ii) worst mutual fund

In [109]:
#finding ytd returns for original portfolio
tickers_worst_ori = original_worst_fund['Ticker Symbol Given by the Exchange']
price_data_worst_1b = yf.download(list(tickers_worst_ori),start = '2021-10-01',end = '2021-10-31')
price_data_worst_ori_1b = price_data_worst_1b['Adj Close']   

[*********************100%***********************]  31 of 31 completed


In [111]:
monthly_worst = pd.DataFrame(tickers_worst_ori)
monthly_worst['monthly_returns']=cal_ytd_returns(tickers_worst_ori, price_data_worst_ori_1b)
monthly_worst['weights'] = weights_worst
monthly_worst['weighted_monthly_returns'] = monthly_worst['monthly_returns']*monthly_worst['weights']

q1b_results.loc[1] = (['worst mutual fund',monthly_worst['weighted_monthly_returns'].sum()*100])
q1b_results

,portfolio,total monthly returns
0,best mutual fund,5.815515
1,worst mutual fund,5.859496


### (iii) portfolio before subjective adjustment

In [116]:
price_data_opt = yf.download(tickers,start = '2021-10-01',end = '2021-10-31')
price_data_opt = price_data_opt['Adj Close']

[*********************100%***********************]  31 of 31 completed


In [120]:
monthly_opt = pd.DataFrame(tickers_worst_ori)
monthly_opt['monthly_returns']=cal_ytd_returns(tickers, price_data_opt)
monthly_opt['weights'] = weights_optimised
monthly_opt['weighted_monthly_returns'] = monthly_opt['monthly_returns']*monthly_opt['weights']

q1b_results.loc[2] = (['portfolio before subjective adjustment',monthly_opt['weighted_monthly_returns'].sum()*100])
q1b_results

,portfolio,total monthly returns
0,best mutual fund,5.815515
1,worst mutual fund,5.859496
2,portfolio before subjective adjustment,6.632408
3,portfolio after subjective adjustment,8.407291


### (iiii) portfolio after subjective adjustment

In [121]:
monthly_opt = pd.DataFrame(tickers_worst_ori)
monthly_opt['monthly_returns']=cal_ytd_returns(tickers, price_data_opt)
monthly_opt['weights'] = weights_adjusted
monthly_opt['weighted_monthly_returns'] = monthly_opt['monthly_returns']*monthly_opt['weights']

q1b_results.loc[3] = (['portfolio after subjective adjustment',monthly_opt['weighted_monthly_returns'].sum()*100])
q1b_results

,portfolio,total monthly returns
0,best mutual fund,5.815515
1,worst mutual fund,5.859496
2,portfolio before subjective adjustment,6.632408
3,portfolio after subjective adjustment,8.407291


The adjusted portfolio had a significant improved (and the best) total monthly returns compared to the best mutual fund's original portfolio and portfolio before subjective adjustment for the month of October 2021. 





In [28]:
price_data_best_ori['date'] = price_data_best_ori.index

In [29]:
price_data_best_ori
df = price_data_best_ori[['ASX', 'date']]

In [54]:
price_data_best_ori['date2'] = price_data_best_ori['date'].dt.strftime('%Y-%m')

In [55]:
price_data_best_ori

,ADI,AEIS,AMAT,AMBA,AMD,AOSL,ASX,AVGO,CRUS,DIOD,...,SEDG,SGH,SMTC,SYNA,TSM,TTMI,TXN,XLNX,date,date2
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-31,145.864197,96.663727,85.829178,91.820000,91.709999,23.639999,5.649126,427.943329,82.199997,70.500000,...,319.119995,37.630001,72.089996,96.400002,107.772926,13.80,160.357758,141.505264,2020-12-01,2020-12
2021-01-04,144.175797,97.680496,86.396065,91.339996,92.300003,24.490000,5.774877,415.599121,84.129997,71.239998,...,311.350006,38.639999,71.180000,97.400002,110.402008,13.75,158.491653,142.164032,2021-01-01,2021-01
2021-01-05,146.752838,101.538284,89.111160,95.250000,92.769997,26.340000,6.113438,418.413940,86.360001,74.019997,...,316.809998,39.560001,72.290001,98.500000,111.459572,14.02,159.654297,143.960663,2021-01-01,2021-01
2021-01-06,147.414383,105.924385,90.334457,102.650002,90.330002,28.000000,6.219842,415.833649,90.379997,77.360001,...,348.130005,39.240002,73.519997,101.269997,114.266579,14.34,160.445679,140.956299,2021-01-01,2021-01
2021-01-07,153.644669,111.177742,94.044106,106.410004,95.160004,28.490000,6.355267,433.358002,91.360001,79.089996,...,365.970001,41.384998,74.650002,102.639999,120.018951,14.28,164.080200,149.430450,2021-01-01,2021-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,180.520004,85.820000,135.759995,179.850006,122.360001,33.240002,6.940000,522.070007,79.370003,94.089996,...,311.920013,51.480000,80.180000,179.399994,113.639999,13.56,198.974518,174.982635,2021-10-01,2021-10
2021-10-26,179.880005,85.279999,132.000000,181.000000,122.930000,32.830002,7.000000,533.869995,79.760002,93.889999,...,313.880005,50.990002,80.540001,178.759995,114.180000,13.69,195.774231,171.559052,2021-10-01,2021-10
2021-10-27,170.460007,84.760002,132.160004,181.690002,122.279999,32.540001,6.930000,523.109985,80.320000,93.540001,...,342.540009,50.680000,81.370003,181.429993,113.750000,13.60,185.954712,180.512299,2021-10-01,2021-10


In [47]:
df=price_data_best_ori.copy(deep=True)

In [48]:
formatted_df = df["date"].dt.strftime("%Y-%m")

In [49]:
formatted_df.head(1

Date
2020-12-31    2020-12
2021-01-04    2021-01
2021-01-05    2021-01
2021-01-06    2021-01
2021-01-07    2021-01
Name: date, dtype: object

In [57]:
formatted_df.iloc[0] =='2020-12'

True

In [43]:
formatted_df['Order Day new']=formatted_df['date'].astype(str)

KeyError: 'date'